In [515]:
import numpy as np
import pandas as pd
import plotly.express as px

In [746]:
# INPUTS
threshold1 = 1
threshold2 = threshold1-0.1
max_tim = 500
leverage = 2
country = {
    'us': 'SPY',
    'france': 'EWQ',
    'germany': 'EWG',
    'uk': 'EWU',
    #'japan': 'EWJ',
    'italy': 'EWI',
    'swiss': 'EWL',
}


In [747]:
bond = pd.read_csv(f'data/IGSB.csv', index_col=0, parse_dates=True)['price']
bond_change = bond.pct_change()

In [757]:
index_dict = dict()
ptf_results = dict()
for name, ticker in country.items():
    price = pd.read_csv(f'data/{ticker}.csv', index_col=0, parse_dates=True)['price']
    index_dict[name] = price
    df = pd.DataFrame()
    df['price'] = price
    df['return'] = price.pct_change()
    df['log_return'] = np.log(price).diff()
    df['rolling_std'] = df['log_return'].rolling(10).std() * np.sqrt(252)
    df['dd'] = df['price'] / df['price'].expanding().max()
    df = df.dropna()
    df['fwd_return'] = df['price'].shift(-252)/df['price'] - 1
    #df = df.dropna()
    #px.scatter(df.set_index('dd')['fwd_return'])
    tim = 0
    ptf = dict()
    units = dict()
    activated = False
    init = False
    for n, index in enumerate(df.index, start=1):
        
        yesterday_index = df.index[n-2]
        today_index = df.index[n-1]

        if init == False:
            init = True
            units[yesterday_index] = 0
            ptf[yesterday_index] = 100_000

        if units[yesterday_index] == 0:
            ptf[today_index] = ptf[yesterday_index] * (1 + bond_change.loc[today_index])
        else:
            ptf[today_index] = ptf[yesterday_index] + units[yesterday_index] * (df.loc[today_index, 'price'] - df.loc[yesterday_index, 'price'])

        if df.loc[index, 'rolling_std'] > threshold1 and activated == False:
            activated = True
            units[today_index] = 0

        elif units[yesterday_index] != 0 and df.loc[index, 'dd'] >= 1:
            units[today_index] = 0

        elif df.loc[index, 'rolling_std'] < threshold2 and activated == True:
            tim = 0
            activated = False
            units[today_index] = (ptf[today_index] / df.loc[index, 'price']) * leverage
        
        else:
            units[today_index] = units[yesterday_index]

        tim += 1
    ptf = pd.Series(ptf).sort_index()
    ptf_results[name] = ptf

In [760]:
index_df = pd.DataFrame(index_dict)
-(((index_df.diff() > 0) * index_df.diff()).sum()) / (((index_df.diff() < 0) * index_df.diff()).sum())

us         1.141434
france     1.041982
germany    1.048433
uk         1.037830
italy      1.022710
swiss      1.076848
dtype: float64

In [763]:
np.exp(np.log(ptf_results.iloc[-1] / ptf_results.iloc[0])/17) - 1

us         0.109429
france     0.138550
germany    0.156947
uk         0.127245
italy      0.073238
swiss      0.093128
dtype: float64

In [765]:
px.line(pd.DataFrame(ptf_results))

In [718]:
price = pd.read_csv(f'data/SPY.csv', index_col=0, parse_dates=True)['price']

In [719]:
px.line(price)

In [521]:
us = pd.read_csv('data/SPY.csv', index_col=0, parse_dates=True)['price']
france = pd.read_csv('data/EWQ.csv', index_col=0, parse_dates=True)['price']
italy = pd.read_csv('data/EWI.csv', index_col=0, parse_dates=True)['price']
germany = pd.read_csv('data/EWG.csv', index_col=0, parse_dates=True)['price']
uk = pd.read_csv('data/EWU.csv', index_col=0, parse_dates=True)['price']
swiss = pd.read_csv('data/EWL.csv', index_col=0, parse_dates=True)['price']

us.name = 'us'
france.name = 'france'
italy.name = 'italy'
germany.name = 'germany'
uk.name = 'uk'
swiss.name = 'swiss'



In [522]:
px.line(france.pct_change().rolling(10).std() * np.sqrt(252))

In [523]:
df = pd.concat([us, france, italy, germany, uk, swiss], axis=1).dropna()
df_log_returns = np.log(df / df.shift(1))
df_rolling_std = df_log_returns.rolling(window=10).std() * np.sqrt(252)
df_leverage = 0.1 / df_rolling_std
df_leverage.iloc[:, :] = 1
df_leverage = df_leverage.dropna()
df = df.loc[df_leverage.index]
df_vt = (df.pct_change() * df_leverage.shift(1)).fillna(0).add(1).cumprod() * 100
df_vt_change = df_vt.pct_change()

df_rolling_1y = df_vt.rolling(window=252*1).apply(lambda x: x.iloc[-1]/x.iloc[0]-1)
df_rolling_1y = df_rolling_1y.dropna()

In [524]:
df_rolling_1y = df_rolling_1y['us']
df_rolling_1y.name = 'return'
df_rolling_std = df_rolling_std['us']
df_rolling_std.name = 'vol'

In [525]:
df = pd.concat([df_rolling_1y, df_rolling_std], axis=1)
df['return'] = df['return'].shift(-251)

In [526]:
df.dropna(inplace=True)

In [527]:
px.scatter(df.set_index('vol')['return'])

In [528]:
res = 200_000*(1+1)**6
print(f'{res:_.0f}')

12_800_000


In [529]:
12_800_000*0.025

320000.0

In [530]:
px.line(df['vol'])

In [531]:
500000*0.9

450000.0

In [532]:
results = {}
for strike in np.linspace(start=0.8, stop=1.2, num=5):
    for option_type in ['call', 'put']:
        premium = BSMModel.compute_option_with_forward(F=1, K=strike, T=1, r=0, sigma=0.1, option_type=option_type)
        if option_type == 'call':
            payoff = df_rolling_1y.map(lambda x: np.max([(100*(1+x)-(strike*100))/100, 0]))
        else:
            payoff = df_rolling_1y.map(lambda x: np.max([((strike*100)-100*(1+x))/100, 0]))
        payoff = payoff.mean(axis=1)
        results[f'{strike}_{option_type}'] = payoff - premium
results = pd.DataFrame(results)

NameError: name 'BSMModel' is not defined

In [ ]:
px.line(results)

In [ ]:
a, b = max_sharpe_weights(results)

In [ ]:
np.round(a*10)

In [ ]:
px.line(results.dot(np.round(a)))

In [ ]:
results.dot(a).mean()*10

In [ ]:
results.loc[:, np.round(a*10)>0]

In [ ]:
(df_rolling_1y).describe()

In [ ]:
df.pct_change().min()

In [ ]:
px.line(df_rolling_1y)

In [ ]:
test_df = pd.DataFrame()

In [ ]:
test_df['spread'] = abs(df_rolling_1y['germany']-df_rolling_1y['france'])

In [ ]:
test_df['germany'] = df_rolling_1y['germany']
test_df['france'] = df_rolling_1y['france']

In [ ]:
px.scatter(test_df.set_index('germany')['france'])

In [ ]:
px.line(df_rolling_1y['germany']-df_rolling_1y['france'])

In [ ]:
# Calculate all possible pairs of correlations
window_size = 252
all_pairs = []
countries = df_vt_change.columns

# Calculate correlations for all unique pairs
for i in range(len(countries)):
    for j in range(i+1, len(countries)):
        pair_corr = df_vt_change[countries[i]].rolling(window=window_size).corr(df_vt_change[countries[j]])
        pair_name = f'{countries[i]}-{countries[j]}'
        all_pairs.append((pair_name, pair_corr))

# Create DataFrame with all correlations
rolling_correlations = pd.DataFrame({pair[0]: pair[1] for pair in all_pairs})

# Plot using plotly
fig = px.line(rolling_correlations, 
              title='1-Year Rolling Correlations of Volatility-Targeted Returns',
              labels={'value': 'Correlation', 'index': 'Date'})
fig.show()

In [ ]:
px.line(rolling_correlations.mean(axis=1))

In [ ]:
hist_correlation = df_vt_change.corr().to_numpy()

simulation_params = {
    "S": [100.0, 100.0, 100.0, 100.0, 100.0],
    "T": 20.0,
    "r": 0.0,
    "g": [0.0, 0.0, 0.0, 0.0, 0.0],
    "q": [0.0, 0.0, 0.0, 0.0, 0.0],
    "sigma": [0.1, 0.1, 0.1, 0.1, 0.1],
    "rho": hist_correlation,
    "n_steps": int(252*20),
    "n_paths": 2000,
    "seed": True,
    "seed_value": 42,
}

paths = MultiAssetBSMModel.compute_montecarlo(**simulation_params)
list(paths.keys())
#paths  = {'asset_0': paths['asset_0']}
final_prices = pd.DataFrame({name: frame.iloc[-1].values for name, frame in paths.items()})
final_prices = final_prices.min(axis=1)
proba = (final_prices < 80).sum() / final_prices.shape[0]
print('proba:', proba)

In [ ]:
n = 5
identity_matrix = np.eye(n)

simulation_params = {
    "S": [100.0, 100.0, 100.0, 100.0, 100.0],
    "T": 5.0,
    "r": 0.0,
    "g": [0.0, 0.0, 0.0, 0.0, 0.0],
    "q": [0.0, 0.0, 0.0, 0.0, 0.0],
    "sigma": [0.1, 0.1, 0.1, 0.1, 0.1],
    "rho": identity_matrix,
    "n_steps": int(252*5),
    "n_paths": 2000,
    "seed": True,
    "seed_value": 42,
}

paths = MultiAssetBSMModel.compute_montecarlo(**simulation_params)
list(paths.keys())
#paths  = {'asset_0': paths['asset_0']}
final_prices = pd.DataFrame({name: frame.iloc[-1].values for name, frame in paths.items()})
final_prices = final_prices.min(axis=1)
proba = (final_prices < 80).sum() / final_prices.shape[0]
print('proba:', proba)

In [ ]:
63/5

In [ ]:
df_vt_change = df_vt_change.dropna()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_vt_change)

# Apply PCA
pca = PCA(n_components=5)
pca_components = pca.fit_transform(scaled_data)

# Create DataFrame with PCA components
pca_df = pd.DataFrame(
    pca_components,
    index=df_vt_change.index,
    columns=[f'PC_{i+1}' for i in range(5)]
)

# Verify correlations are 0
print("PCA components correlation matrix:")
print(pca_df.corr().round(3))

# Show explained variance ratio
print("\nExplained variance ratio:")
print(pca.explained_variance_ratio_)

In [ ]:
pca_df = pca_df * (0.1 / (pca_df.std() * np.sqrt(252)))

In [ ]:
px.line(pca_df.add(1).cumprod())